# MNIST training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [4]:
import os
import json

import sagemaker
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role


sess = sagemaker.Session()

role = get_execution_role()

output_path='s3://' + sess.default_bucket() + '/mxnet/mnist'

## MXNet Estimator

The `MXNet` class allows you to run your training script on SageMaker
infrastracture in a containerized environment. In this notebook, we will
refer to this container as *training container*. 

You need to configure
it with the following parameters to set up the environment:

- entry_point: A user defined python file to be used by the training container as the 
instructions for training. We will further discuss this file in the next subsection.

- role: An IAM role to make AWS service requests

- instance_type: The type of SageMaker instance to run your training script. 
Set it to `local` if you want to run the training job on 
the SageMaker instance you are using to run this notebook

- instance count: The number of instances you need to run your training job. 
Multiple instances are needed for distributed training.

- output_path: 
S3 location to save training output (model artifacts and output files)

- framework_version: The version of MXNet you need to use.

- py_version: The python version you need to use

For more information, see [the API reference](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)



## Implement the entry point for training

The entry point for training is a python script that provides all 
the code for training a MXNet model. It is used by the SageMaker 
MXNet Estimator (`MXNet` class above) as the entry point for running the training job.

Under the hood, SageMaker MXNet Estimator downloads a docker image
with runtime environemnts 
specified by the parameters you used to initiated the
estimator class and it injects the training script into the 
docker image to be used as the entry point to run the container.

In the rest of the notebook, we use *training image* to refer to the 
docker image specified by the MXNet Estimator and *training container*
to refer to the container that runs the training image. 

This means your training script is very similar to a training script
you might run outside Amazon SageMaker, but it can access the useful environment 
variables provided by the training image. Checkout [the short list of environment variables provided by the SageMaker service](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/using_mxnet.html?highlight=entry%20point) to see some common environment 
variables you might used. Checkout [the complete list of environment variables](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md) for a complete 
description of all environment variables your training script
can access to. 

In this example, we will use the training script `code/train.py`
as the entry point for our MXNet Estimator.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [ ]:
!pygmentize 'code/train.py'

### Set hyperparameters

In addition, MXNet estimator allows you to parse command line arguments
to your training script via `hyperparameters`.

<span style="color:red"> Note: local mode is not supported in SageMaker Studio </span>

In [10]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode=True

if local_mode:
    instance_type='local'
else:
    instance_type='ml.c4.xlarge'
    
est = MXNet(
    entry_point='train.py',
    source_dir='code', # directory of your training script
    role=role,
    framework_version='1.7.0',
    py_version='py3',
    instance_type=instance_type,
    instance_count=1,
    output_path=output_path,
    hyperparameters={
        'batch-size':100,
        'epochs':20,
        'learning-rate': 0.1,
        'momentum': 0.9,
        'log-interval':100
    }
)


The training container executes your training script like

```
python train.py --batch-size 100 --epochs 20 --learning-rate 0.1
    --momentum 0.9 --log-interval 100
```

## Set up channels for training and testing data

You need to tell `MXNet` estimator where to find your training and 
testing data. It can be a link to an S3 bucket or it can be a path
in your local file system if you use local mode. In this example,
we download the MNIST data from a public S3 bucket and upload it 
to your default bucket. 

In [8]:
import logging
import boto3
from botocore.exceptions import ClientError


# Download training and testing data from a public S3 bucket


def download_from_s3(data_dir='/tmp/data', train=True):
    """Download MNIST dataset and convert it to numpy array
    
    Args:
        data_dir (str): directory to save the data
        train (bool): download training set
    
    Returns:
        None
    """
    
    # Get global config
    with open('code/config.json', 'r') as f:
        CONFIG=json.load(f)
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client('s3')
    bucket = CONFIG['public_bucket']
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return


download_from_s3('/tmp/data', True)
download_from_s3('/tmp/data', False)


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    Args:
        file_name (str): File to upload
        bucket (str): Bucket to upload to
        object_name (str): S3 object name. If not specified then file_name is used
        
    Return:
        True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

mnist_data = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz", 
    "t10k-labels-idx1-ubyte.gz"
]

for f in mnist_data:
    fpath = os.path.join('/tmp/data', f)
    dest = os.path.join('mxnet/mnist', f)
    upload_file(fpath, sess.default_bucket(), dest)
    
    
# set up channels for training and testing data
train_data_loc = "s3://" + sess.default_bucket() + '/mxnet/mnist'
test_data_loc = "s3://" + sess.default_bucket() + '/mxnet/mnist'

channels={
    'training': train_data_loc,
    'testing': test_data_loc
}


The keys of the dictionary `channels` are parsed to the training image
and it creates the environment variable `SM_CHANNEL_<key name>`. 

In this example, `SM_CHANNEL_TRAINING` and `SM_CHANNEL_TESTING` are created in the training image (checkout 
how `code/train.py` access these variables). For more information,
see: [SM_CHANNEL_{channel_name}](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md#sm_channel_channel_name)

If you want, you can create a channel for validation:
```
channels = {
    'training': train_data_loc,
    'validation': val_data_loc,
    'test': test_data_loc
    }
```
You can then access this channel within your training script via
`SM_CHANNEL_VALIDATION`


## Run the training script on SageMaker
Now, the training container has everything to execute your training
script. You can start the container by calling `fit` method.

In [11]:
est.fit(inputs=channels)

Using the short-lived AWS credentials found in session. They might expire while running.


Creating tmpx9pcz29a_algo-1-13waq_1 ... 
Attaching to tmpx9pcz29a_algo-1-13waq_12mdone
algo-1-13waq_1  | 2020-11-21 01:38:06,652 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
algo-1-13waq_1  | 2020-11-21 01:38:06,654 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-13waq_1  | 2020-11-21 01:38:06,665 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-13waq"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"batch-size":100,"epochs":20,"learning-rate":0.1,"log-interval":100,"momentum":0.9}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-13waq","hosts":["algo-1-13waq"]}', 'SM_INPUT_DATA_CONFIG': '{"testing":{"TrainingInputMode":"File"},"training":{"TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '["testing","training"]', 'SM_CURRENT_HOST': 'algo-1-13waq',

algo-1-13waq_1  | [Epoch 0 Batch 100] Training: accuracy=0.807723, 35275.895711 samples/s
algo-1-13waq_1  | [Epoch 0 Batch 200] Training: accuracy=0.862935, 36734.139079 samples/s
algo-1-13waq_1  | [Epoch 0 Batch 300] Training: accuracy=0.887010, 30094.740618 samples/s
algo-1-13waq_1  | [Epoch 0 Batch 400] Training: accuracy=0.899800, 36589.932827 samples/s
algo-1-13waq_1  | [Epoch 0 Batch 500] Training: accuracy=0.910499, 35891.699469 samples/s
algo-1-13waq_1  | [Epoch 0] Training: accuracy=0.917917
algo-1-13waq_1  | [Epoch 0] Validation: accuracy=0.958000
algo-1-13waq_1  | [Epoch 1 Batch 100] Training: accuracy=0.962970, 30801.968128 samples/s
algo-1-13waq_1  | [Epoch 1 Batch 200] Training: accuracy=0.960746, 35575.097540 samples/s
algo-1-13waq_1  | [Epoch 1 Batch 300] Training: accuracy=0.961927, 30639.959091 samples/s
algo-1-13waq_1  | [Epoch 1 Batch 400] Training: accuracy=0.962843, 35092.904953 samples/s
algo-1-13waq_1  | [Epoch 1 Batch 500] Training: accuracy=0.963693, 29916.576

algo-1-13waq_1  | [Epoch 13 Batch 200] Training: accuracy=0.991642, 34447.306176 samples/s
algo-1-13waq_1  | [Epoch 13 Batch 300] Training: accuracy=0.991163, 34798.838463 samples/s
algo-1-13waq_1  | [Epoch 13 Batch 400] Training: accuracy=0.991372, 35353.202967 samples/s
algo-1-13waq_1  | [Epoch 13 Batch 500] Training: accuracy=0.991297, 34827.733953 samples/s
algo-1-13waq_1  | [Epoch 13] Training: accuracy=0.991450
algo-1-13waq_1  | [Epoch 13] Validation: accuracy=0.996367
algo-1-13waq_1  | [Epoch 14 Batch 100] Training: accuracy=0.996040, 33600.128174 samples/s
algo-1-13waq_1  | [Epoch 14 Batch 200] Training: accuracy=0.996318, 28368.643896 samples/s
algo-1-13waq_1  | [Epoch 14 Batch 300] Training: accuracy=0.996146, 28110.073051 samples/s
algo-1-13waq_1  | [Epoch 14 Batch 400] Training: accuracy=0.996185, 34790.179164 samples/s
algo-1-13waq_1  | [Epoch 14 Batch 500] Training: accuracy=0.995788, 35237.368731 samples/s
algo-1-13waq_1  | [Epoch 14] Training: accuracy=0.995817
algo-1-1

## Inspect and store model data

Now, the training is finished, the model artifact has been saved in 
the `output_path`. We 

In [12]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)

Model artifact saved at:
 s3://sagemaker-us-west-2-688520471316/mxnet/mnist/mxnet-training-2020-11-21-01-38-01-009/model.tar.gz


We will store the variable `model_data` in the current notebook kernel. 
In the [next notebook](get_started_with_mnist_deploy.ipynb), you will learn how to retrieve the model artifact and deploy to a SageMaker
Endpoint.

In [13]:
%store model_data

Stored 'model_data' (str)


## Test and debug the entry point before executing the training container

The entry point `code/train.py` provided here has been tested and it can be executed in the training container. 
When you do develop your own training script, it is a good practice to simulate the container environment 
in the local shell and test it before sending it to SageMaker, because debugging in a containerized environment
is rather cumbersome. The following script shows how you can test your training script:

In [ ]:
!pygmentize code/test_train.py